In [1]:
## NAME:Devyani Mardia
## NETID: dm1633
## Group members: David & Devyani


################
## Code for HW 3 problem 1
##
## INSTRUCTIONS:
##
## The following file implements Kernel Ridge Regression with
## Gaussian (RBF) Kernel.
##
## It uses KRR to predict the housing price on the Boston
## Housing Data-set, where each row is a small neighborhood
## in Boston and where the features describe the environment
## of the neighborhood.
##
## Fill in the code in parts labeled "FILL IN".
## There are 4 parts where you have to fill in.
##
##
################

import numpy as np
import pandas as pd

np.random.seed(1)

houses = pd.read_csv("Boston.csv")

features = houses.columns.values
features = features[np.logical_not(np.isin(features, ["X", "medv"]))]

# Response variable
Y = houses["medv"].to_numpy()

# Features
X = houses[features].to_numpy()

# Normalize X (so units are standardized)
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

n = 300

test_ixs = np.random.choice(houses.shape[0], houses.shape[0] - n, replace=False)
learn_ixs = [x for x in range(houses.shape[0]) if x not in test_ixs]

Xlearn = X[learn_ixs, :]
Ylearn = Y[learn_ixs]

Xtest = X[test_ixs, :]
Ytest = Y[test_ixs]

In [2]:
## Cross validation to select bandwidth and lambda

nfold = 5
fold_mat = np.arange(n).reshape(int(n/nfold), nfold)

## We have to select both bandwidth and regularization parameter
lambda_ls = np.concatenate(([0], float(2) ** np.arange(-8, 7, 1)))
bandwidth_ls = float(2) ** np.arange(-6, 7, 1)

valid_errs = np.zeros((nfold, len(bandwidth_ls), len(lambda_ls)))

for k in range(nfold):
    valid_ixs = fold_mat[:, k]
    train_ixs = [x for x in range(Xlearn.shape[0]) if x not in valid_ixs]

    Xtrain = Xlearn[train_ixs, :]
    Ytrain = Ylearn[train_ixs]

    Xvalid = Xlearn[valid_ixs, :]
    Yvalid = Ylearn[valid_ixs]

    ## computes all pairwise distances between data points in Xtrain
    xtx = Xtrain @ Xtrain.T
    xnorms = np.diag(xtx)
    train_dists = np.ones((Xtrain.shape[0], 1)) @ xnorms.reshape(1, -1) - 2*xtx + xnorms.reshape(-1, 1) @ np.ones((1, Xtrain.shape[0]))
    ## train_dists[i, i'] is the distance | Xtrain[i, :] - Xtrain[i', :] |^2

    ## computes all pairwise distances between data points in Xvalid and Xtrain
    xtx2 = Xvalid @ Xtrain.T
    xnorms_valid = np.diag(Xvalid @ Xvalid.T)
    train_valid_dists = np.ones((Xvalid.shape[0], 1)) @ xnorms.reshape(1, -1) - 2*xtx2 + xnorms_valid.reshape(-1, 1) @ np.ones((1, Xtrain.shape[0]))
    ## train_valid_dists[i, i'] is the distance | Xvalid[i, :] - Xtrain[i', :] |^2

    for j in range(len(bandwidth_ls)):
        h = bandwidth_ls[j]

        ## K[i, i'] contains K( Xtrain[i, :], Xtrain[i', :] )
        K = np.exp(-train_dists / (h ** 2))

        ## K2[i, i'] contains K( Xvalid[i, :], Xtrain[i', :] )
        K2 = np.exp(-train_valid_dists / (h ** 2))

        for l in range(len(lambda_ls)):
            lambda_ = lambda_ls[l]

            alpha = np.linalg.inv(   K + (lambda_ * np.identity(len(K)) )  ) @ Ytrain
            ## Ypred[i] should be predicted value for Xvalid[i]

            Ypred = np.zeros(len(Xvalid))
            for v in range(len(Xvalid)):
                val = 0
                for m in range(len(Xtrain)):
                    val = val + K2[v,m]*alpha[m]
                Ypred[v] = val
            # Ypred =  val ## FILL IN; see definition of "K2"
            
            valid_errs[k, j, l] = np.mean((Yvalid - Ypred)**2)
            
mean_valid_errs = np.apply_over_axes(np.mean, valid_errs, axes=(0,))


In [3]:
best_ix = np.argmin(mean_valid_errs, axis=None)
best_ix2 = np.unravel_index(best_ix, mean_valid_errs.shape)

hstar = bandwidth_ls[best_ix2[1]]
lambda_star = lambda_ls[best_ix2[2]]

print(f"Selected bandwidth: {hstar:.3f}   lambda: {lambda_star:.3f}")

Selected bandwidth: 8.000   lambda: 0.004


In [4]:
## computes all pairwise distances between
## data points in Xlearn
xtx = Xlearn @ Xlearn.T
xnorms_learn = np.diag(xtx)
xlearn_dists = np.ones((Xlearn.shape[0], 1)) @ xnorms_learn.reshape(1, -1) - 2*xtx + xnorms_learn.reshape(-1, 1) @ np.ones((1, Xlearn.shape[0]))
## xlearn_dists[i, i'] is the distance | Xlearn[i, ] - Xlearn[i', ] |^2


## computes all pairwise distances between data points in
## Xtest and Xlearn
xtx2 = Xtest @ Xlearn.T
xnorms_test = np.diag(Xtest @ Xtest.T)
xlearn_test_dists = np.ones((Xtest.shape[0], 1)) @ xnorms_learn.reshape(1, -1) - 2*xtx2 + xnorms_test.reshape((-1, 1)) @ np.ones((1, Xlearn.shape[0]))
## xlearn_test_dists[i, i'] is the distance | Xtest[i, ] - Xlearn[i', ] |^2

## K[i, i'] contains K( Xlearn[i, ], Xlearn[i', ])
K = np.exp(-xlearn_dists / (hstar** 2) )

## K2[i, i'] contains K( Xtest[i, ], Xlearn[i', ])
K2 = np.exp( -xlearn_test_dists / (hstar ** 2) )

lambda_ = lambda_star

alpha = np.linalg.inv(   K + (lambda_ * np.identity(len(K)) )  ) @ Ylearn
## Ypred[i] should be predicted value for Xvalid[i]

Ypred = np.zeros(len(Xtest))
for v in range(len(Xtest)):
    val = 0
    for m in range(len(Xlearn)):
        val = val + K2[v,m]*alpha[m]
    Ypred[v] = val

test_err = np.mean((Ytest - Ypred)**2)

## Compute the MSE for OLS for comparison purpose
Xlearn_a = np.concatenate((Xlearn, np.ones((Xlearn.shape[0], 1))), axis=1)
Xtest_a = np.concatenate((Xtest, np.ones((Xtest.shape[0], 1))), axis=1)

betahat = np.linalg.solve(Xlearn_a.T @ Xlearn_a, Xlearn_a.T @ Ylearn)
Ypred2 = Xtest_a @ betahat
test_err2 = np.mean((Ytest - Ypred2)**2)


print("KRR MSE: %.3f   OLS MSE: %.3f" % (test_err, test_err2))
print("KRR Test R-squared: %.3f   OLS Test R-squared: %.3f" % (1 - test_err/np.var(Ytest), 1 - test_err2/np.var(Ytest)))

KRR MSE: 11.240   OLS MSE: 24.836
KRR Test R-squared: 0.874   OLS Test R-squared: 0.723


In [5]:
mean_valid_errs

array([[[570.1088    , 570.1088    , 570.1088    , 570.1088    ,
         570.1088    , 570.1088    , 570.1088    , 570.1088    ,
         570.1088    , 570.1088    , 570.1088    , 570.1088    ,
         570.1088    , 570.1088    , 570.1088    , 570.1088    ],
        [570.1088    , 570.1088    , 570.1088    , 570.1088    ,
         570.1088    , 570.1088    , 570.1088    , 570.1088    ,
         570.1088    , 570.1088    , 570.1088    , 570.1088    ,
         570.1088    , 570.1088    , 570.1088    , 570.1088    ],
        [570.10879165, 570.10879168, 570.10879172, 570.10879178,
         570.1087919 , 570.10879214, 570.10879258, 570.10879332,
         570.10879443, 570.10879583, 570.10879722, 570.10879833,
         570.10879907, 570.10879951, 570.10879975, 570.10879987],
        [569.63100277, 569.63284096, 569.63466505, 569.63827162,
         569.64532264, 569.65880923, 569.68355745, 569.72569722,
         569.78906698, 569.86854945, 569.94833234, 570.01237507,
         570.05517712,

Part b (free response)
Examine the value of the matrix mean_valid_errs, whose (j; k)-th entry is the mean (across
folds) validation error of bandwidth j and lambda k among the candidate sets bandwidth_ls and
lambda_ls.

Use your observation to justify the importance of selecting both the bandwidth h and the
regularization parameter   in cross-validation.


Solution: 

When we perform cross-validation, we examine all potential lambda-bandwidth pairs and select the set that provides us with optimal model performance. If we didn’t, we would likely create a model using sub-standard parameters, which would likely yield a poor model. It is also important that we examined both parameters together during cross-validation, as they do not influence the performance of the model independently. That is, we cannot first select the “best” lambda and use that model to then select the 
“best” bandwidth (or vice versa); they must be vali-dated in tandem.
Looking at the mean_valid_errs list generated in this problem, we can see that cross validation allowed us to select from parameter candidates that yielded substantially varied error. Selecting our optimal lambda-bandwidth pair helped us reduce our error tenfold.